In [11]:

import os
from IPython.display import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import pysd
from pysd.builders.stan.stan_model import StanVensimModel
from pysd.translators.vensim.vensim_file import VensimFile
from pysd.translators.xmile.xmile_file import XmileFile
import matplotlib.pyplot as plt

import cmdstanpy # 2.30 is fastest (as of 08.12.2022) `cmdstanpy.install_cmdstan()` 
from cmdstanpy import CmdStanModel, cmdstan_path
import arviz as az #!pip install git+https://github.com/arviz-devs/arviz
az.style.use("arviz-darkgrid")


# set your working directiory
#os.chdir("/Users/hyunjimoon/GoogleDrive_hmb/pysd/test_scripts")

In [2]:
obs_stock_df = pd.read_csv('data/hudson-bay-lynx-hare.csv')
vf = VensimFile("vensim_models/prey-predator.mdl")
vf.parse()
am = vf.get_abstract_model()

# set time
n_t = obs_stock_df.shape[0] - 1
premodel = StanVensimModel("prey-predator", am, 0.0, list(range(1, n_t + 1)))
premodel.print_info()

n_t = obs_stock_df.shape[0] - 1
data_data2draws = {
    "n_t": n_t,
    "predator_obs": obs_stock_df.loc[1:, 'Predator'].values.tolist(),
    "prey_obs": obs_stock_df.loc[1:, 'Prey'].values.tolist(),
}

- Vensim model information:
original name       stan variable name  is stock
------------------------------------------------
delta               delta               
predator birth rate predator_birth_rate 
gamma               gamma               
predator death rate predator_death_rate 
Predator            predator            V
alpha               alpha               
prey birth rate     prey_birth_rate     
beta                beta                
prey death rate     prey_death_rate     
Prey                prey                V
FINAL TIME          final_time          
INITIAL TIME        initial_time        
SAVEPER             saveper             
TIME STEP           time_step           
**********
- Stan model information:


In [4]:

premodel = StanVensimModel("prey-predator", am, 0.0, list(range(1, n_t + 1)), data_dict = data_data2draws)
# ode parameter 
premodel.set_prior("alpha", "normal", 0.55, 0.1)
premodel.set_prior("gamma", "normal", 0.8, 0.1)
premodel.set_prior("beta", "normal", 0.028, 0.01)
premodel.set_prior("delta", "normal", 0.024, 0.01)

# sampling distribution parameter
premodel.set_prior("sigma", "normal", 0, 1)

premodel.set_prior("predator_obs", "lognormal", "predator", "sigma")
premodel.set_prior("prey_obs", "lognormal", "prey", "sigma")
premodel.build_stan_functions()
draws2data_model = premodel.draws2data()
#print(premodel.vensim_model_context.variable_names)


#cmdstan_model = premodel.data2draws(data_data2draws)
#result = cmdstan_model.sample(data=data_data2draws)
#result.summary()

prey-predator_functions.stan already exists in the current working directory. Overwrite? (Y/N):y


21:35:13 - cmdstanpy - INFO - compiling stan file /Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data.stan to exe file /Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data


['alpha', 'gamma', 'beta', 'delta', 'sigma', 'predator_obs', 'prey_obs']


21:35:16 - cmdstanpy - INFO - compiled model executable: /Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data
21:35:16 - cmdstanpy - WARNING - Stan compiler has produced 1 warnings:
21:35:16 - cmdstanpy - WARNING - 
--- Translating Stan model to C++ code ---
bin/stanc --include-paths=/Users/dashadower/git_repos/pysd/test_scripts/stan_files --o=/Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data.hpp /Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data.stan
Warning in '/Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data.stan', line 29, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc

--- Compiling, linking C++ code ---
clang++ -std=c++1y -Wno-unknown-warning-option -

In [10]:
pd.set_option('display.max_rows', 500)
draws2data_model = CmdStanModel(stan_file="stan_files/prey-predator_draws2data.stan")
draws2data_model.sample(data=data_data2draws, fixed_param=True).summary()

21:41:27 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

21:41:27 - cmdstanpy - INFO - CmdStan done processing.


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
alpha,5.500000e-01,NaN,2.554790e-15,5.500000e-01,5.500000e-01,5.500000e-01,NaN,NaN,NaN
gamma,8.000000e-01,NaN,3.110180e-15,8.000000e-01,8.000000e-01,8.000000e-01,NaN,NaN,NaN
beta,2.800000e-02,NaN,6.248130e-17,2.800000e-02,2.800000e-02,2.800000e-02,NaN,NaN,NaN
delta,2.400000e-02,NaN,5.901010e-17,2.400000e-02,2.400000e-02,2.400000e-02,NaN,NaN,NaN
sigma,1.000000e-08,NaN,2.648300e-23,1.000000e-08,1.000000e-08,1.000000e-08,NaN,NaN,NaN
predator__init,4.000000e+00,NaN,2.665870e-15,4.000000e+00,4.000000e+00,4.000000e+00,NaN,NaN,NaN
prey__init,3.000000e+01,NaN,7.108980e-14,3.000000e+01,3.000000e+01,3.000000e+01,NaN,NaN,NaN
initial_outcome[1],4.000000e+00,NaN,2.600000e-15,4.000000e+00,4.000000e+00,4.000000e+00,NaN,NaN,NaN
initial_outcome[2],3.000000e+01,NaN,7.100000e-14,3.000000e+01,3.000000e+01,3.000000e+01,NaN,NaN,NaN
"integrated_result[1,1]",4.435270e+00,NaN,3.500000e-15,4.435270e+00,4.435270e+00,4.435270e+00,NaN,NaN,NaN


In [ ]:
import pandas as pd
vensim_df = pd.read_csv("vensim_models/prey-predator/output.csv")
predator_obs = vensim_df[vensim_df['Time']=="Predator"]
prey_obs = vensim_df[vensim_df['Time']=="Prey"]
print(predator_obs)
print(prey_obs)

In [6]:
draws2data_model.sample(data=data_data2draws).summary()

21:35:47 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

21:35:48 - cmdstanpy - INFO - CmdStan done processing.
21:35:48 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lognormal_rng: Scale parameter is -0.35927, but must be positive finite! (in '/Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data.stan', line 33, column 4 to column 72)
	Exception: lognormal_rng: Scale parameter is -0.0993442, but must be positive finite! (in '/Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data.stan', line 33, column 4 to column 72)
	Exception: lognormal_rng: Scale parameter is -0.660059, but must be positive finite! (in '/Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data.stan', line 33, column 4 to column 72)
	Exception: lognormal_rng: Scale parameter is -0.281241, but must be positive finite! (in '/Users/dashadower/git_repos/pysd/test_scripts/stan_files/prey-predator_draws2data.stan', line 33, column 4 to column 72)
	Exception: lognormal_rng: Scale

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
alpha,NaN,NaN,NaN,NaN,NaN,6.234340e-01,NaN,NaN,NaN
gamma,NaN,NaN,NaN,NaN,0.757316,NaN,NaN,NaN,NaN
beta,NaN,NaN,NaN,NaN,0.038457,NaN,NaN,NaN,NaN
delta,NaN,NaN,NaN,NaN,0.023717,NaN,NaN,NaN,NaN
sigma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
prey_obs[16],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
prey_obs[17],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
prey_obs[18],NaN,NaN,NaN,6.598890e+11,23699.000000,NaN,NaN,NaN,NaN
prey_obs[19],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
premodel.draws2data(data_data2draws)

# Structuring Uncertainties in Dynamic Models: 
## Bayesian workflow of Predator-Prey Population Dynamics

Angie.H Moon, 07.2022

## Data: Predator and Prey Pelts in Canada

The species of interest in this case study are

- hares: prey, an hervivorous cousin of rabbits, and
- lynxes: predator, a feline predator whose diet consists largely of hares.

Spikes in the predator population lag those in the prey population. When populations are plotted against one another over time, the population dynamics orbit in an apparently stable pattern. Population oscillations can be modeled with a pair of differential equations similar to that used to describe springs. The first plot is the number of lynx and hare pelts (in thousands) collected for twenty years. The second plot is the phase plot of number of pelts collected for lynx versus hares similar to that of the dynamics of a spring in phase space (i.e., position vs. momentum).

In [ ]:
obs_stock_df = pd.read_csv('data/hudson-bay-lynx-hare.csv')
# data viz
pd.melt(obs_stock_df, id_vars = 'Year').iloc[[0,20,21,41]]
pd.melt(obs_stock_df, id_vars = 'Year').iloc[[0,1,20,21,40,41]].rename(columns = {'variable':'species', 'value':'pelts in thousands'})
ax = obs_stock_df.loc[:, ['Predator', 'Prey']].plot()
ax.set(xlabel='year', ylabel='pelt (thousands)') 

Phase plots are as below:

In [ ]:
plt.scatter(obs_stock_df.loc[:, 'Predator'], obs_stock_df.loc[:, 'Prey'])
plt.plot(obs_stock_df.loc[:, 'Predator'], obs_stock_df.loc[:, 'Prey'])
plt.xlabel('Predator pelts (thousands)')
plt.ylabel('Prey pelts (thousands)')

## Mechanistic Model: The Lotka-Volterra Equations

In Lotka-Volterra equations (Lotka 1925; Volterra 1926, 1927), Volterra modeled the temporal dynamics of the two species (i.e., population sizes over times) in terms of four parameters, $\alpha, \beta, \gamma, \delta \geq 0$, as

$$
\begin{eqnarray}
\frac{\mathrm{d}}{\mathrm{d}t} u
& = &  (\alpha - \beta v) u
& = & \alpha u - \beta u v
\\[6pt]
\frac{\mathrm{d}}{\mathrm{d}t} v
& = &  (-\gamma + \delta \, u) \, v
& = & -\gamma v + \delta uv
\end{eqnarray}
$$

$u(t)$ and $v(t)$ are rendered as $u$ and $v$.  The factor $\alpha$, $\beta$ are the rate of birth and shrinkage relative to the product of the population sizes where as $\gamma$, $\delta$  are the shrinkage and growth rate as a factor of the product of the population sizes. Both u and v have positivitity constraints. as long as the initial populations are non-negative, i.e., $u(0) \geq 0$ and $v(0) \geq 0$, because the rate of change in each population is a factor of the population size itself.

The dynamic system has four limiting behavior:

1. If both population sizes are initially positive, the populations will oscillate in a fixed pattern indefinitely, remaining positive.
2. If both population sizes are initially zero, the population sizes will remain zero.
3. If the predator population size is zero and the prey population size positive, the predator population size remains zero and the prey population grows without bound.
4. If the predator population size is positive and the prey population size zero, the prey population size remains zero while the predator population shrinks toward zero size.

## Statistical Model: regreasion framing and uncertainty embedding

### Solving the inverse problem
Bayesian statistics is somewhat counterintuitive, as it involves formulating the generating model (from parameter to observed data) then using general principles to solve the inverse problem. Specifically, a Bayesian model requires a mathematical model of what we know about the parameters (i.e., a prior) and a model of what we know about the data generating process given the parameters (i.e., a sampling distribution.

Mathematically, a prior density $p(\theta)$ over the sequence of parameters $\theta$ encapsulates our knowledge of the parameters before seeing the data.  A sampling distribution (or likelihood), which may have a continuous, discrete or mixed probability function, $p(y | \theta)$ characterizes the distribution of observable data $y$ given parameters $\theta$. We limit the observation as stock variables as every SD model can be reformulated into the combination of stock and parameters.

Bayes's rule gives us a general solution to the inverse problem, expressing the posterior $p(\theta | y)$ in terms of the prior $p(\theta)$ and likelihood $p(y | \theta)$. Stan provides a form of Markov chain Monte Carlo (MCMC) sampling that draws a sample $\theta^{(1)}, \ldots, \theta^{(M)}$ from the posterior to use for computational inference. Posterior quantities of interest may be expressed as derived random variables using functions $f(\theta)$ of parameters. This feature is used for decision analysis; for instance, imagine a optimization problem of conservation cost of the park where prey and predator ecology places at. The cost can be computed based on the posterior distribution inferred from the observed time series.


### Uncertainty embedding for forward-backward symmetry required for calibration

The Lotka-Volterra model is deterministic in that given the value of the system parameter and initial outcome state, equation solutions (simulated outcome value) are fully determined. However, for empirical research which use posterior inference from the real data as it final forecast, forward model should be re-designed. This is because symmetry of forward and backward model (i.e. data generation and its inference) is the theoretical justification of calibration. To pass this internal consistency test (or with enough resource, SBC which is rank-statistics based), we need the two process to be the mirror image of other. This is why we purposefully embed uncertainty components, waiting to be captured in the inference step. The purpose is to test resilience and identifiability of our models evidenced by the perfect retrival of prior distribution for every uncertainty we embedded. 

### Linear regression analogy

Like in a simple linear regression, we will proceed by treating the underlying determinstic model as providing an expected population value around which there will be variation due to both measurement error and simplifications in the scientific model. Consider the typical formulation of a linear regression, where $y_n$ is an observable scalar outcome, $x_n$ is a row vector of unmodeled predictors (aka covariates, features), $\beta$ is a coefficient vector parameter, and $\sigma > 0$ is the error scale,

$$
\begin{eqnarray}
y_n & = & x_n \beta + \epsilon_n
\\[6pt]
\epsilon_n & \sim & \mathsf{Normal}(0, \sigma)
\end{eqnarray}
$$

### Adding measurement uncertainty (epistemic)
Before embedding parameteric uncertainty, linear predictor $x_n \beta$ with predictor $x_n$ (row $n$ of the data matrix $x$) and coefficient (column) vector $\beta$ are deterministic. The only source of uncertainty is from the measurement. This is expressed by assigning a normal distribution to error term $\epsilon_n$. Equal expression is with latent error variable $\epsilon_n$ as follows[<sup>17</sup>](#fn17), 

$$
y_n \sim \mathsf{Normal}(x_n \beta, \sigma).
$$

### Adding parameter uncertainty (epistemic)
Next, we add parameter uncertainty by coding estimated parameter as a distribution rather than a fixed value. This distribution is called prior distribution and from our example, Normal distirbution is chosen to endow the uncertainty to the four estimated parameters $\alpha, \beta, \gamma, \delta$. Considering their role difference, $\alpha, \gamma$ as multipliers of $u, -v$ and $\beta, \delta$ as multipliers of $uv$, prior parameter are chosen as N(1, 0.5) and N(0.05, 0.05) for each. For this selection, refer to the original case study [Carpenter18](https://mc-stan.org/users/documentation/case-studies/lotka-volterra-predator-prey.html).


### Adding aleatoric uncertainty
This applies only when we decide to add measurement uncertainty i.e. having initial value of stock variable as `est_param` instead of the default `ass_param`. For this, detailed explanation is added at the end of this file at Appendix A. 

## Prior predictive check

In [ ]:
vf = VensimFile("vensim_models/prey-predator.mdl")
vf.parse()
am = vf.get_abstract_model()

# set time
n_t = obs_stock_df.shape[0] - 1
premodel = StanVensimModel("prey-predator", am, 0.0, list(range(1, n_t + 1)))
premodel.print_info()

In [ ]:
n_t = obs_stock_df.shape[0] - 1
data_data2draws = {
    "n_obs_state" : 2,
    "initial_time" : 0, 
    "times": [i+1 for i in np.arange(n_t)],
    "n_t": n_t,
    "y": obs_stock_df.loc[1:, ('Predator', 'Prey')].values.tolist(),  # (predator, prey)
}

In [ ]:


# ode parameter 
premodel.set_prior("alpha", "normal", 0.55, 0.1)
premodel.set_prior("gamma", "normal", 0.8, 0.1)
premodel.set_prior("beta", "normal", 0.028, 0.01)
premodel.set_prior("delta", "normal", 0.024, 0.01)

# sampling distribution parameter
premodel.set_prior("sigma", "normal", 0, 1)
premodel.set_prior("y[:,1]", "lognormal", "predator", "sigma")
premodel.set_prior("y[:,2]", "lognormal", "prey", "sigma")

premodel.build_stan_functions()
#premodel.draws2data()
#print(premodel.vensim_model_context.variable_names)
model = StanVensimModel("prey-predator", am, 0.0, list(range(1, n_t + 1)))
model.set_prior("sigma", "normal", 0, 1)
model.set_prior("y[:,1]", "lognormal", "predator", "sigma")
model.set_prior("y[:,2]", "lognormal", "prey", "sigma")

cmdstan_model = model.data2draws({})
result = cmdstan_model.sample()
result.summary()

In [ ]:
data_draws2data = {
    "n_obs_state" : 2
}

sf_path_draws2data = os.path.join(os.getcwd(), "stan_file", "prey-predator_draws2data.stan")
sm_draws2data = CmdStanModel(stan_file = sf_path_draws2data)

prior_pred = sm_draws2data.sample(data=data_draws2data, iter_sampling=3, chains=1, fixed_param=True, iter_warmup=0)
prior_pred.summary()

For prior predictive checks, if the real observed data is indiscriminable from the simulated, we usually view it as a sign of pass. Real data is an external reference so as long as the predicted ranges are not too off, we give a pass to prior predictive check. Summary statistics such as N^th moments can be used for comparison. Few comments:

a. we use real data below as a representation of our knowledge, so prior predictive check is not double dipping (using data twice)

b. Bayesian prior corresponds to frequentist's regularization so having a tighter prior than posterior is not unnatrual; simply our determination to find a model concentrated around certain model configuration

c. if tight prior is well-placed, it prevents diveregence from frustrating geometry and boosts sampling efficiency

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
#compare with real 
ax.plot(obs_stock_df.loc[:, ['Prey']], label = "Real_Pey")
ax.plot(obs_stock_df.loc[:, ['Predator']], label = "Real_Predator")
ax.plot(pd.DataFrame(prior_pred.y_tilde[:,:,0]).T.loc[:, :5])
ax.plot(pd.DataFrame(prior_pred.y_tilde[:,:,1]).T.loc[:, :5])
ax.legend()

## Posterior predictive check

Now we estimate parameter values based on the real observed stock values. Here we use more diffuse prior.

In [ ]:
vf = VensimFile("vensim_models/prey-predator.mdl")
vf.parse()
am = vf.get_abstract_model()

# set time
model = StanVensimModel("prey-predator", am, 0.0, list(range(1, n_t + 1)))

# ode parameter 
model.set_prior("alpha", "normal", 0.8, 0.1)
model.set_prior("gamma", "normal", 0.8, 0.1)
model.set_prior("beta", "normal", 0.05, 0.001)
model.set_prior("delta", "normal", 0.05, 0.001)

# sampling distribution parameter
# model.set_prior("sigma", "lognormal", np.log(0.01), 0.1) 

print(model.vensim_model_context.variable_names)

model.build_stan_functions()
model.draws2data("")
model.data2draws("")

In [ ]:
sf_path_data2draws = os.path.join(os.getcwd(), "stan_file", "prey-predator_data2draws.stan")
sm_data2draws = CmdStanModel(stan_file = sf_path_data2draws)

n_t = obs_stock_df.shape[0] - 1
data_data2draws = {
    "n_obs_state" : 2,
    "initial_time" : 0, 
    "times": [i+1 for i in np.arange(n_t)],
    "n_t": n_t,
    "y": obs_stock_df.loc[1:, ('Predator', 'Prey')].values.tolist(),
}

fit = sm_data2draws.sample(data = data_data2draws, iter_sampling = 100, chains = 1, show_console = False, seed = 1234)

The following is the summary of posterior draws. It includes loglikelihood for each vector of parmaeter values $(\alpha, \beta, \gamma, \delta, \sigma)$. 

In [ ]:
fit.summary()

In [ ]:
idata = az.from_cmdstanpy(
    posterior=fit, 
    posterior_predictive=["y_hat"], 
    log_likelihood= ["log_lik"],
    observed_data = {"y": obs_stock_df.loc[:, ("Predator", "Prey")]}
#    dtypes={"y_rep": int} if Poisson family
)

In [ ]:
az.loo(idata)
az.plot_ppc(idata, alpha=0.03, figsize=(12, 6))

## Appendix A. Measurement error, when stock initial is estimated parameter

Lastly, initial population is added to the list of estimated parameter. Instant reason for this modeling decision is measurement noise; as population cannot be directly measured pelts (our data) are used as its noisy proxy). However, more fundamental reason is aletoric uncertainty, namely unmodeled uncertainty. There are factors that impact predator and prey population size other than the current population size. There are variable environmental effects, such as weather, which will vary from season to season and year to year and affect population sizes. Infectious diseases occasionally spread through a population, reducing its size (Hewitt 1921). There are also more long-term environmental factors such as carrying capacity (Carrying capacity is roughly the maximum population that an environment can sustain. It is often modeled in the system dynamics as an asymptote on population size.). However, our forward model is restricted to two differential equations involving two stock variables (`simulated outcome`) and four `estimated coefficient parameter`s (four flow variables can be expressed with the other two). Hence, after restricting the architecture, we are reaching out to the best version of ourselve by declaring the initial population as estimated parameter. In Stat/Machine learning terms, Stan optimization algorithm returns `estimated parameter` value that maximize log posterior among the feasible (restricted basis function) space defined by the modeler in the form of stock-parameter relationship.

Continuing on `simulated outcome` and `observed outcome` coflow, `observed outcome` can replace `simulated outcome` also known as state-resetting but we maintain the error term to compensate for measurement error and unexplained variation in the data (Challenge: check whether this is equivalent to the original text "Solutions to the Lotka-Volterra equations replace the linear predictor xnβ, but we maintain the error term to compensate for measurement error and unexplained variation in the data."). In the case of population dynamics, the data $y_n$ consists of measurements of the prey $y_{n, 1}$ and predator $y_{n, 2}$ populations at times $t_n$[<sup>18</sup>](#fn18).

The true population sizes at time $t = 0$ are unknown---we only have measurements $y^{\rm init}_1$ and $y^{\rm init}_2$ of them. The true initial population sizes at time $t = 0$ will be represented by a parameter $z^{\mathrm init}$, so that

$$
\begin{array}{rcl}
z^{\mathrm init}_1 & = & u(t = 0)
\\[4pt]
z^{\mathrm init}_2 & = & v(t = 0).
\end{array}
$$

Next, let $z_1, \ldots, z_N$ be the solutions to the Lotka-Volterra differential equations at times $t_1, \ldots, t_N$ given initial conditions $z(t = 0) = z^{\mathrm init}$ and parameters $\theta = (\alpha, \beta, \gamma, \delta)$. Each $z_n$ is a pair of prey and predator population sizes at the specified times,

$$
\begin{array}{rcl}
z_{n, 1} & = & u(t_n)
\\[4pt]
z_{n, 2} & = &  v(t_n).
\end{array}
$$

The $z_n$ are random variables, but they are deterministic functions of the random variables for the initial state $z^{\mathrm init}$ and system parameters $\alpha, \beta, \gamma, \delta$.

The observed data is in the form of measurements $y^{\rm init}$ of the initial population of prey and predators, and subsequent measurements $y_n$ of the populations at times $t_n$, where $y^{\mathrm init}$ and the $y_n$ consist of a pair of measured population sizes, for the prey and predator species.

In summary, the measurements, $y^{\rm init}$ and $y_n$, are drawn indepently from a normal distribution centered at the underlying population sizes, $z^{\rm init}$ and $z_n$, with noise scales $\sigma$.




## Appendix B. Stan code

The following is the auto-generated stanfile with modularized funtion block in the cell below.
```
functions{
#include prey-predator_functions.stan
}
data{
    int <lower = 1> n_obs_state;
    real initial_time;
    int <lower = 1> n_t; 
    array[n_t] real times;
    array[n_t] vector[n_obs_state] y; 
}


parameters{
    real alpha;
    real gamma;
    real beta;
    real delta;
    real sigma;
}

transformed parameters {
    # Initial ODE values
    real prey_initial = 30;
    real predator_initial = 4;

    vector[2] initial_outcome;  # Initial ODE state vector
    initial_outcome[1] = prey_initial;
    initial_outcome[2] = predator_initial;

    vector[2] integrated_result[n_t] = ode_rk45(vensim_func, initial_outcome, initial_time, times, alpha, beta, gamma, delta, sigma);
}

model{
    alpha ~ normal(0.8, 0.1);
    gamma ~ normal(0.8, 0.1);
    beta ~ normal(0.05, 0.001);
    delta ~ normal(0.05, 0.001);
    sigma ~ lognormal(-4.605170185988091, 1);
    for (s in 1: n_obs_state){
        y[:, s] ~ lognormal(log(integrated_result[:, s]), sigma);
    }
}

generated quantities{
    array[n_t] vector[n_obs_state] y_ppc;
    vector[n_t] log_lik;
    for (s in 1: n_obs_state){
        y_ppc[:, s] = lognormal_rng(log(integrated_result[:, s]), sigma);
    } 
    for (s in 1: n_obs_state){
        //elementwise log likliehood
        log_lik[s] = lognormal_lpdf(y[s]|log(integrated_result[:, s]), sigma);
    }
}
```

Function block:
```
# Begin ODE declaration
vector vensim_func(real time, vector outcome, real alpha, real beta, real gamma, real delta){
    vector[2] dydt;  # Return vector of the ODE function

    # State variables
    real prey = outcome[1];
    real predator = outcome[2];

    real prey_birth_rate = alpha * prey;
    real predator_birth_rate = delta * prey * predator;
    real prey_death_rate = beta * predator * prey;
    real prey_dydt = prey_birth_rate - prey_death_rate;
    real predator_death_rate = gamma * predator;
    real predator_dydt = predator_birth_rate - predator_death_rate;

    dydt[1] = prey_dydt;
    dydt[2] = predator_dydt;

    return dydt;
}


```